<a href="https://colab.research.google.com/github/dbasrai/watchmyset/blob/master/WMS_predict_YT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install pydub
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import ntpath
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!python -m pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose
!pip install python_speech_features
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank

def gen_chunks(audio_filename, output_folder):
  myaudio = AudioSegment.from_file(audio_filename) 
  chunk_length_ms = 1000 # pydub calculates in millisec
  chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
  chunk_list = []

  base_filename = ntpath.basename(audio_filename)

  if len(base_filename) > 7:
    vid_title = base_filename[0:6]
  else:
    vid_title = base_filename

  #Export all of the individual chunks as wav files

  for i, chunk in enumerate(chunks):
      chunk_name = f"{vid_title}chunk{i}.wav"
      print ("exporting", chunk_name)
      chunk.export(f'{output_folder}/{chunk_name}', format="wav")
      chunk_list.append(chunk_name)
  
  chunk_list.pop()

  return vid_title, chunk_list

def delete_path(path):
  for root, dirs, files in os.walk(path):
      for f in files:
          os.unlink(os.path.join(root, f))
      for d in dirs:
          shutil.rmtree(os.path.join(root, d))

In [0]:
!pip install youtube_dl
from __future__ import unicode_literals
import youtube_dl
import os

output_path = '/content/gdrive/My Drive/Colab Notebooks/youtube_dl_wav'
chunk_path = '/content/gdrive/My Drive/Colab Notebooks/chunks'
yt_url = 'https://www.youtube.com/watch?v=bCoJduAhbgs'

ydl_opts = {
    'outtmpl': os.path.join(output_path, '%(title)s-%(id)s.%(ext)s'),
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192'
    }],
    'postprocessor_args': [
        '-ar', '16000'
    ],
    'prefer_ffmpeg': True,
    'keepvideo': True
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
  ydl.download([yt_url])

for filename in os.listdir(output_path):
    if filename.endswith(".wav"):
      title, chunkname_list = gen_chunks(f'{output_path}/{filename}', chunk_path)

delete_path(output_path)

    

In [0]:
import librosa
from keras.models import load_model
from sklearn.metrics import accuracy_score

X_pred_list = []
_min, _max = float('inf'), -float('inf')

model_name = 'WMS_model_5.5k.model'
model = load_model(f'/content/gdrive/My Drive/Colab Notebooks/models/{model_name}')

for i in tqdm(chunkname_list):
  signal, rate = librosa.load(f'{chunk_path}/{i}', sr=16000)
  mel = mfcc(signal[:rate], rate, numcep=13, nfilt=26, nfft=512)
  X_pred_list.append(mel)
  _min = min(np.amin(X_pred_list), _min)
  _max = max(np.amax(X_pred_list), _max)

X_pred = np.array(X_pred_list)
X_pred = (X_pred - _min) / (_max - _min)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[1], X_pred.shape[2])

y_pred = model.predict(X_pred)

y_guess = np.argmax(y_pred, axis=1)

np.savetxt(f"/content/gdrive/My Drive/Colab Notebooks/predictions/{model_name}_{title}_csv_predict.csv", y_guess, delimiter=",")
np.savetxt(f"/content/gdrive/My Drive/Colab Notebooks/predictions/{model_name}_{title}_csv_probability.csv", y_pred, delimiter=",")



Using TensorFlow backend.
100%|██████████| 339/339 [00:51<00:00,  6.61it/s]


In [0]:
import math as math
from matplotlib.pyplot import step, show
import matplotlib.pyplot as plt

numLaughter = np.count_nonzero(y_guess)
laughPercent =  round((numLaughter / y_guess.size) * 100, 1)
laughTimes = []
laughTimesList = []
numLaughs = 0
check = 0

for i in tqdm(range(y_guess.size)):
  if (y_guess[i]==1 and check==0):
    numLaughs=numLaughs+1
    check = 1
    laughTimes.append(i)
  else:
    check = y_guess[i]

for i in tqdm(range(len(laughTimes))):
  seconds = np.mod(laughTimes[i], 60)
  minutes = np.floor_divide(laughTimes[i], 60)
  laughTimesList.append('{0} minutes '.format(minutes) + 'and {0} seconds'.format(seconds))

laughsPerMin = numLaughs / (y_guess.size/60)

print('\n{0} stats'.format(title))
print('Laugh Percentage = {0}%'.format(laughPercent))
print('Num Laughs = {0}'.format(numLaughs))
print('Laughs Per Minute (LPM) = {0}'.format(round(laughsPerMin, 1)))
print("\n".join(laughTimesList))

plt.rcParams["figure.figsize"] = (50,10)
plt.title('Laughs Over Time')
xaxis = np.arange(0, y_guess.shape[0])
plt.xticks(np.arange(min(xaxis), max(xaxis)+1, 5.0))
frame1 = plt.gca()
frame1.axes.get_yaxis().set_visible(False)
yaxis = y_guess
step(xaxis, yaxis)
plt.plot
plt.savefig(f'/content/gdrive/My Drive/Colab Notebooks/plots/{title}_.png', dpi=100)